In [1]:
from pyspark.sql import SparkSession

# Create SparkSession from builder
# If the sample data you work with is small, you can remove the `.config` call
spark = SparkSession.builder.appName('Spark') \
            .config("spark.driver.memory", "10g") \
            .getOrCreate()
sc = spark.sparkContext

23/04/14 11:09:59 WARN Utils: Your hostname, Mufins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.245.5 instead (on interface en0)
23/04/14 11:09:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/14 11:09:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/14 11:09:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
user_df = spark.read.json('user.json')
user_df.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- description: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- description: struct (nullable = true)
 |    |    |-- cashtags: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end: long (nullable = true)
 |    |    |    |    |-- start: long (nullable = true)
 |    |    |    |    |-- tag: string (nullable = true)
 |    |    |-- hashtags: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end: long (nullable = true)
 |    |    |    |    |-- start: long (nullable = true)
 |    |    |    |    |-- tag: string (nullable = true)
 |    |    |-- mentions: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end: long (nullable = true)
 |    |    |    |    |-- start: long (nullable = true)
 |    |    |    |    |-- username: string (nullable = true)
 |    |

In [9]:
user_df.select("id").show(10)

+--------------------+
|                  id|
+--------------------+
|u1217628182611927040|
|         u2664730894|
|u1266703520205549568|
|u1089159225148882949|
|           u36741729|
|u1365527332627247104|
|         u1679822588|
|         u1519144464|
|           u15211869|
|u1309034737756000256|
+--------------------+
only showing top 10 rows



In [10]:
user_filter_df = spark.read.csv('sample_user.csv', header=True)
user_df = user_filter_df.join(user_df, on="id", how="left")

In [11]:
user_df.show()

+--------------------+--------------------+------------------------------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------+--------------------+--------------------+---------------+--------+--------+
|                  id|          created_at|                         description|            entities|            location|                name|    pinned_tweet_id|   profile_image_url|protected|      public_metrics|                 url|       username|verified|withheld|
+--------------------+--------------------+------------------------------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------+--------------------+--------------------+---------------+--------+--------+
|u1052241356574330880|2018-10-16 16:54:...|                             podador|                null|Lloret de Vistale...|               Nadal|               null|https://abs.twimg...|   

In [12]:
import pyspark.sql.functions as F

user_df = user_df.withColumn('created_at_timestamp', F.to_date('created_at')) \
                .withColumn('age', F.datediff(F.current_date(), 'created_at'))
user_df = user_df.withColumn('tweet_frequency', user_df['public_metrics.tweet_count'] / user_df['age']) \
                .withColumn('followers_growth_rate', user_df['public_metrics.followers_count'] / user_df['age']) \
                .withColumn('following_growth_rate', user_df['public_metrics.following_count'] / user_df['age']) \
                .withColumn('listed_growth_rate', user_df['public_metrics.listed_count'] / user_df['age']) \
                .withColumn('followers_following_ratio', user_df['public_metrics.followers_count'] / F.when(user_df['public_metrics.following_count'] == 0, 1).otherwise(user_df['public_metrics.following_count'])) \
                .withColumn('name_length', F.length('name')) \
                .withColumn('num_digits_in_name', F.length(F.regexp_extract('name', r'\d+', 0))) \
                .withColumn('username_length', F.length('username')) \
                .withColumn('num_digits_in_username', F.length(F.regexp_extract('username', r'\d+', 0)))
                # .withColumn('description_length', F.length('description'))

extracted_df = user_df.select([
    'public_metrics.followers_count',
    'public_metrics.following_count',
    'public_metrics.listed_count',
    'public_metrics.tweet_count',
    'protected',
    'verified',
    'age',
    'tweet_frequency',
    'followers_growth_rate',
    'following_growth_rate',
    'followers_following_ratio',
    'name_length',
    'num_digits_in_name',
    'username_length',
    'num_digits_in_username'
])
# Cast each column to IntegerType
for c in extracted_df.columns:
    extracted_df = extracted_df.withColumn(c, F.col(c).cast("int"))

In [13]:
from pyspark.ml.feature import StandardScaler, VectorAssembler

# Convert columns to a vector column
assembler = VectorAssembler(inputCols=extracted_df.columns, outputCol="features")
vectorized_df = assembler.transform(extracted_df)

# Standardize the vector column
scaler = StandardScaler(inputCol="features", outputCol="scaled_features",
                        withStd=True, withMean=True)
scalerModel = scaler.fit(vectorized_df)
scaled_df = scalerModel.transform(vectorized_df)

# Get mean and standard deviation
mean = scalerModel.mean.toArray()
std = scalerModel.std.toArray()

# Print mean and standard deviation
print("Mean: ", mean)
print("Standard Deviation: ", std)

23/04/14 11:12:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Mean:  [1.27236530e+05 3.01896906e+03 4.69078113e+02 2.15224633e+04
 7.73395205e-04 2.02629544e-01 3.69836350e+03 5.47563805e+00
 2.80696056e+01 6.00928074e-01 1.00828770e+03 1.46674401e+01
 4.40835267e-02 1.09474091e+01 3.63495746e-01]
Standard Deviation:  [2.01244548e+06 1.26720187e+04 2.61769171e+03 4.76464280e+04
 2.78099839e-02 4.02114246e-01 1.50535105e+03 1.40102926e+01
 3.93984832e+02 3.44439228e+00 1.69221065e+04 7.28600801e+00
 3.36719788e-01 2.71368847e+00 1.21941118e+00]


In [14]:
import torch

featuresArray = scaled_df.select("scaled_features").rdd.flatMap(lambda x: x).collect()
tfidfTensor = torch.tensor(featuresArray)
tfidfTensor.shape

torch.Size([1293, 15])

In [ ]:
class VariedDataset(torch.utils.data.Dataset):
    
    def __init__(self, torch_list):
        self.torch_list = torch_list
        
    def __getitem__(self, idx):
        if idx >= len(self):
            raise ValueError(f"Index exceeds length of the dataset of {len(self)}")
        return self.torch_list[idx]
    
    def __len__(self):
        return len(self.torch_list)

In [15]:
from torch.utils.data import TensorDataset

dataset = VariedDataset(tfidfTensor)

In [19]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=8)
torch.save(dataloader, "dataloaders/sample/user.pt")

In [20]:
# Get mean and standard deviation
mean = scalerModel.mean.toArray()
std = scalerModel.std.toArray()

In [21]:
type(mean)

numpy.ndarray

In [22]:
# Save mean and std
import numpy as np

np.save("dataloaders/sample/user_mean.npy", mean)
np.save("dataloaders/sample/user_std.npy", std)

In [26]:
label_df = spark.read.csv('label.csv', header=True)
label_df = label_df.withColumn("label_bool", F.col("label") == "bot")
label_df = label_df.withColumn("label_int", F.col("label_bool").cast("integer"))
label_df = label_df.select(F.col("id"), F.col("label_int").alias("label"))
label_df.show(5)

+--------------------+-----+
|                  id|label|
+--------------------+-----+
|u1217628182611927040|    0|
|         u2664730894|    0|
|u1266703520205549568|    0|
|u1089159225148882949|    0|
|           u36741729|    1|
+--------------------+-----+
only showing top 5 rows



In [27]:
label_df = user_filter_df.join(label_df, on="id", how="left")
label_df.show(5)

+--------------------+-----+
|                  id|label|
+--------------------+-----+
|         u1468355888|    0|
|           u25493114|    0|
|         u2967455656|    0|
|          u145637907|    0|
|u1403349174893096965|    0|
+--------------------+-----+
only showing top 5 rows



In [29]:
label_df.filter(label_df["label"].isNotNull()).count()

1293

In [31]:
labelArray = label_df.select("label").rdd.flatMap(lambda x: x).collect()
len(labelArray)

1293

In [32]:
labelTensor = torch.tensor(labelArray)
dataset = VariedDataset(labelTensor)
dataloader = DataLoader(dataset, batch_size=8)
torch.save(dataloader, "dataloaders/sample/user_labels.pt")